Expandir contrações

Exemplo: I'll -> I will 

resultado: dataset_expandido.csv

In [30]:
import pandas as pd
import contractions

# Lê o dataset
df = pd.read_csv("dataset_final_limpo.csv")

# Função para expandir abreviações
def expand_contractions(text):
    if pd.isna(text):
        return text
    return contractions.fix(text)

# Aplica a expansão na coluna "text"
df['text'] = df['text'].apply(expand_contractions)

# Salva o resultado
df.to_csv("dataset_expandido.csv", index=False)

print("Concluído! Contrações expandidas salvas em 'dataset_expandido.csv'.")


Concluído! Contrações expandidas salvas em 'dataset_expandido.csv'.


remover menções

@gabriel

resultado: dataset_sem_mencoes.csv

In [31]:
#remover mentions
import re

df = pd.read_csv("dataset_expandido.csv")

#remover apenas @
def remove_mentions(text):
    if pd.isna(text):
        return text
    return re.sub(r'@', '', text)

df['text'] = df['text'].apply(remove_mentions)

# Salva o resultado
df.to_csv("dataset_sem_mencoes.csv", index=False)


remover #

#mentalhealth -> mentalhealth

resultado: dataset_sem_hashtags.csv

In [14]:
#remove #

df = pd.read_csv("dataset_sem_mencoes.csv")

def remove_hashtags(text):
    if pd.isna(text):
        return text
    return re.sub(r'#', '', text)

df['text'] = df['text'].apply(remove_hashtags)

# Salva o resultado
df.to_csv("dataset_sem_hashtags.csv", index=False)

Separação de palavras juntas

Exemplo: mentalheath -> mental health

resultado: dataset_palavras_separadas.csv

In [20]:
import wordninja
import pandas as pd

df = pd.read_csv("dataset_sem_hashtags.csv")

def separar_palavras(text):
    if pd.isnull(text):
        return ""

    linhas = text.splitlines()  # preserva as quebras de linha
    linhas_processadas = []

    for linha in linhas:
        words = []
        for word in linha.split():
            if len(word) > 6 and word.isalpha():
                split = wordninja.split(word)
                words.extend(split if len(split) > 1 else [word])
            else:
                words.append(word)
        linha_processada = ' '.join(words)
        linhas_processadas.append(linha_processada)

    return '\n'.join(linhas_processadas)  # junta novamente com \n

df['text'] = df['text'].astype(str).apply(separar_palavras)

df.to_csv("dataset_palavras_separadas.csv", index=False)


Remoção de URLs

resultado: dataset_sem_urls.csv

In [29]:
import pandas as pd
import re

# Suponha que você já tenha carregado seu dataset
# Exemplo de carregamento (ajuste o caminho conforme necessário)
df = pd.read_csv('dataset_palavras_separadas.csv')

def remove_urls(text):
    # Remove padrões comuns de URLs: http(s):// ou www.
    return re.sub(r'https?://\S+|www\.\S+', '', text)

# Aplica a função apenas na coluna 'text'
df['text'] = df['text'].astype(str).apply(remove_urls)

# Salva o resultado em um novo arquivo CSV
df.to_csv('dataset_sem_urls.csv', index=False)
print("Concluído! URLs removidas e salvas em 'dataset_sem_urls.csv'.")


Concluído! URLs removidas e salvas em 'dataset_sem_urls.csv'.


Remoção de abreviações

Exemplo: kms ->  kill myself

resultado: dataset_abreviacoes.csv

In [32]:
abbreviations = {
    "afaik": "as far as I know",
    "afk": "away from keyboard",
    "asap": "as soon as possible",
    "atm": "at the moment",
    "b4": "before",
    "b/c": "because",
    "bcs": "because",
    "brb": "be right back",
    "btw": "by the way",
    "bff": "best friend forever",
    "dm": "direct message",
    "fml": "fuck my life",
    "fomo": "fear of missing out",
    "ftw": "for the win",
    "gg": "good game",
    "g2g": "got to go",
    "gtg": "got to go",
    "hbu": "how about you",
    "hmu": "hit me up",
    "idc": "I don't care",
    "idk": "I don't know",
    "ikr": "I know right",
    "ily": "I love you",
    "imho": "in my humble opinion",
    "imo": "in my opinion",
    "irl": "in real life",
    "jsyk": "just so you know",
    "jk": "just kidding",
    "k": "okay",
    "kms": "kill myself",
    "lmk": "let me know",
    "lol": "laughing out loud",
    "lmao": "laughing my ass off",
    "nvm": "never mind",
    "omg": "oh my god",
    "omw": "on my way",
    "rn": "right now",
    "rofl": "rolling on the floor laughing",
    "smh": "shaking my head",
    "tbh": "to be honest",
    "tfw": "that feeling when",
    "tho": "though",
    "thx": "thanks",
    "ttyl": "talk to you later",
    "ty": "thank you",
    "wbu": "what about you",
    "wdym": "what do you mean",
    "wth": "what the hell",
    "wtf": "what the fuck",
    "yw": "you’re welcome",
    "imfine": "I'm fine",  # frequentemente usado sarcasticamente
    "idek": "I don't even know"
}


In [38]:
df = pd.read_csv('dataset_sem_urls.csv')

import string

def expand_abbreviations_preserve_newlines(text, abbr_dict):
    if pd.isnull(text):
        return ""
    
    lines = text.split('\n')
    expanded_lines = []

    for line in lines:
        words = line.split()
        expanded = []

        for word in words:
            # Remove pontuação das bordas da palavra, mas guarda ela para restaurar depois
            prefix = ''
            suffix = ''

            while len(word) > 0 and word[0] in string.punctuation:
                prefix += word[0]
                word = word[1:]

            while len(word) > 0 and word[-1] in string.punctuation:
                suffix = word[-1] + suffix
                word = word[:-1]

            # Expande se estiver no dicionário
            expanded_word = abbr_dict.get(word.lower(), word)

            # Junta novamente com a pontuação original
            expanded.append(prefix + expanded_word + suffix)

        expanded_lines.append(' '.join(expanded))

    return '\n'.join(expanded_lines)



df['text'] = df['text'].apply(lambda x: expand_abbreviations_preserve_newlines(x, abbreviations))


# Salva o resultado em um novo arquivo CSV
df.to_csv('dataset_abreviacoes.csv', index=False)



TESTE

In [28]:
import pandas as pd
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.dicts.emoticons import emoticons

# Carregue seu dataset
df = pd.read_csv("dataset_sem_urls.csv")

# Configuração do Ekphrasis
text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user', 'time', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated", 'emphasis', 'censored'},
    fix_html=True,
    segmenter="twitter",  # funciona bem com textos informais, mesmo fora do Twitter
    corrector="twitter",  # corrige palavras comuns com erro
    unpack_hashtags=False,  # se quiser separar hashtags como #mentalhealth → mental health
    unpack_contractions=True,  # don't → do not
    spell_correct_elong=False,
    tokenizer=None,
    dicts=[emoticons]  # expande emojis/emoticons como ":)" → "smile"
)

# Função para processar um texto
def process_text(text):
    if pd.isnull(text):
        return ""
    return " ".join(text_processor.pre_process_doc(text))

# Aplica no DataFrame
df['text'] = df['text'].astype(str).apply(process_text)

# Salva o novo CSV
df.to_csv("dataset_ekphrasis.csv", index=False)


Reading twitter - 1grams ...


/home/rolla/.local/lib/python3.10/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in
